# Bitcoin Transaction Network Analysis - Data Pipeline

In [53]:
import blocksci
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import multiprocessing as mp
import itertools
import random
import string
import pickle
import gc
import os, sys
from functools import partial
%matplotlib notebook
ncpu=mp.cpu_count()
print('Number of CPUs: {}'.format(ncpu))

Number of CPUs: 8


In [54]:
gc.collect()

132

In [22]:
# Point to parsed blockchain data
chain = blocksci.Blockchain("/home/ubuntu/bitcoin")
types=blocksci.address_type.types
total_blocks=chain.blocks
print('Total Blocks up to {}:  {} '.format(total_blocks[-1].time,len(total_blocks)))

Total Blocks up to 2019-04-17 16:57:37:  572046 


## Network Definition

In [69]:
#---SCRIPT: generates data for graphs in each part of the partition

start='2015-02-01 00:00:00'
end='2017-02-01 11:59:59'
blocks=chain.range(start=start,end=end)
sample_size=200
partition=BchainPartition(chain,start,end,sample_size=sample_size)
df,graphs=partition_data(partition)
df.head()

# Save to Disk
# Generate an extraction ID
extraction_id = ''.join([random.choice(string.ascii_letters + string.digits) for n in range(6)])



Start_block: 341392
End_block: 451049
Number of Blocks: 109658 
Highest block height: 451049
Number of Transactions: 200 
Number of parts: 200
419842 Processed
449541 Processed
396806 Processed
419336 Processed
394156 Processed
444939 Processed
421900 Processed
377346 Processed
357370 Processed
438190 Processed
376342 Processed
433177 Processed
358778 Processed
449029 Processed
412195 Processed
374820 Processed
363047 Processed
406060 Processed
405557 Processed
424504 Processed
408927 Processed
386110 Processed
436800 Processed
416833 Processed
352835 Processed
436662 Processed
353863 Processed
392264 Processed
449036 Processed
436663 Processed
369740 Processed
411213 Processed
374350 Processed
397905 Processed
429497 Processed
355419 Processed
419421 Processed
374878 Processed
427104 Processed
379067 Processed
365670 Processed
344167 Processed
362089 Processed
426090 Processed
351339 Processed
397420 Processed
413294 Processed
416367 Processed
436849 Processed
434363 Processed
355445 

FileNotFoundError: [Errno 2] No such file or directory: 'extractions/15Tx02-200-blocks-2015_02_01_00_00_00-2017_02_01_11_59_59/15Tx02-200-blocks-2015_02_01_00_00_00-2017_02_01_11_59_59.csv'

In [70]:
#---Save Dataframes

# Create directory and save

start_c=start
start_c=start_c.replace('-','_').replace(' ','_').replace(':','_')
end_c=end
end_c=end_c.replace('-','_').replace(' ','_').replace(':','_')

directory='extractions/'+extraction_id+'-'+str(sample_size)+'-blocks-'+start_c+'-'+end_c+'/graphs'+'/'


if not os.path.exists(directory):
    os.makedirs(directory)

# Create Filename and save

filename='extractions/'+extraction_id+'-'+str(sample_size)+'-blocks-'+start_c+'-'+end_c+'/'+extraction_id+'-'+str(sample_size)+'-blocks-'+start_c+'-'+end_c+'.csv'
df.to_csv(filename)

# Graphs
for g in graphs:
    nx.write_gpickle(g,directory+str(g.graph['graph_id'])+'.gpickle')
    #print('Graph: {} saved'.fromat(g.graph['graph_id']))

print('\n***EXTRACTION COMPLETED SUCCESSFULLY***')


***EXTRACTION COMPLETED SUCCESSFULLY***


In [39]:
# Class that creates a blockchain a blockchain partition (dictionary) given data range and partition type (blocks,days,weeks)

class BchainPartition():
    
    def __init__(self,chain,start_timestamp,end_timestamp,ptype='blocks',sample_size=10):
        blocks=chain.range(start=start_timestamp,end=end_timestamp)
        
        
        self.block_h=blocks.height
        print('Start_block: {}'.format(self.block_h[0]))
        print('End_block: {}'.format(self.block_h[-1]))
        
        if sample_size>0: #Samples blocks from the 
            sample_list=list(np.random.choice(self.block_h,sample_size))
            sample_blocks=[chain[ix_b] for ix_b in sample_list]
            txs=[b.txes for b in sample_blocks]
            self.partition={h:[t for t in t_l] for h,t_l in zip(sample_list,txs)}
            self.no_parts=len(sample_blocks)
            
        else:  
            if ptype=='blocks':
                self.partition={b.height:[tx for tx in b.txes] for b in blocks}
                self.no_parts=np.int32(len(blocks))
                
        
        print('Number of Blocks: {} '.format(len(blocks)))
        print('Highest block height: {}'.format(blocks[-1].height))
        print('Number of Transactions: {} '.format(len(txs)))

        # ***TODO: Create partition for other types of partitions (use tx.block_time)
    

In [65]:
# Function that takes blockchain partition and outputs pandas data frame with features 
# for the graph defined by each split in the partition

def partition_data(chainpartiton):
    
    # Dictionary with partition
    partition=chainpartiton.partition
    partindex=partition.keys()
    parts=partition.values()
    data_tuples=[]
    graphs=[]
    
    print('Number of parts: {}'.format(len(partindex)))
    
    tuples=[(index,part) for index,part in zip(partindex,parts)]
    
    for t in tuples:
        
        data_i,columns_i,graph_i=graph_features(t,slice_type='blocks')
        data_tuples.append((data_i,columns_i))
        graphs.append(graph_i)
         
    
    '''    
    chunksize=len(tuples)%ncpu
    with mp.Pool(processes=ncpu) as pool:
        data_tuples=pool.map(graph_features,tuples,chunksize)
    
    '''
    
    columns=data_tuples[0][1] #This value is being re-written. This design choice is to mantain consistency with columns.
    data=[i for i,j in data_tuples]
    data=np.array(data)    
    df=pd.DataFrame(data=data[:,:],columns=columns)
    
    
    return (df,graphs)




In [41]:
# Function that receives a chain part (list of transactions), generates transaction graph and calculates statistics

def graph_features(chain_part_tuple,slice_type='blocks'):
    
    index=chain_part_tuple[0]
    chain_part=chain_part_tuple[1]
    block_height=chain_part[-1].block_height
    graph=block_graph(chain_part,index,slice_type)
    nx.info(graph)
    nodes=graph.nodes(data=True)
    edges=graph.edges(data=True)
    data=[index]  
    columns=['block_height']
    
    # Number of Nodes
    no_nodes=nx.number_of_nodes(graph)
    data.append(no_nodes)
    columns.append('no_nodes')
    
    # Number of Edges (address to address transactions)
    no_edges=nx.number_of_edges(graph)
    data.append(no_edges)
    columns.append('no_edges')
    
    # Total value transacted
    total_value=np.sum(np.array([a['value'] for n1,n2,a in edges]))
    data.append(total_value)
    columns.append('value_transacted')
    
    # Total Density
    density=nx.density(graph)
    data.append(density)
    columns.append('total_density')
    
    # Nodes with self loops nx.loops nodes_with_selfloops(G) nodes_with_selfloops(G) 
    nodes_self=nx.number_of_selfloops(graph)
    data.append(nodes_self)
    columns.append('nodes_self')
    
    # Value of self loops nodes_with_selfloops(G)
    values=np.array([a['value'] for n1,n2,a in nx.selfloop_edges(graph,data=True)])
    selfloop_value=np.sum(values)
    data.append(selfloop_value)
    columns.append('selfloop_value')
    
    # Number of transactions to old addresses 
    
    old_nodes=[n for n,a in nodes if a['block_created']<block_height]
    edges_to_old=graph.in_edges(old_nodes,data=True)
    data.append(len(edges_to_old))
    columns.append('old_nodes_in')
    
    # Ratio of transactions to old addresses to total transactions
    
    ratio_oldin_totalin=len(edges_to_old)/(no_edges+1)
    data.append(ratio_oldin_totalin)
    columns.append('ratio_oldin_totalin')
    
    # Value of transactions to old addresses
    
    value_to_old=[a['value'] for n1,n2,a in edges_to_old]
    data.append(np.sum(np.array(value_to_old)))
    columns.append('value_to_old')
    
    # Old address density
    
    old_graph=nx.induced_subgraph(graph,old_nodes)
    old_density=nx.density(old_graph)
    data.append(old_density)
    columns.append('old_density')
    
    
    # ***TODO*** (Aggregated graph analysis)
    
    # Accumulated reuse
    # Dominance (Agg graph or new vs. old dominance) 
    #https://networkx.github.io/documentation/stable/reference/algorithms/dominance.html
    # Common ancenstors (as with dominance the address ancestor path should be proportional 
    #to the blockchain lenght if address reuse is minimal)
    #***********
    
    print('{} Processed'.format(index))
    return (data,columns,graph)

In [42]:
# Function that creates transaction graph for a given number transactions


def block_graph(txs,index,slice_type):

    
    # Create graph and process 
    graph = nx.MultiDiGraph(graph_id=index,slice_type=slice_type)
    nodes=[]
    edges=[]
    
    
    # Extract transactions information
    
    init_block=txs[0].block.height
    txs_dic={tx.index:tx for tx in txs}
    txs_ix=list(txs_dic.keys())
    txs_ix.sort()
   
    
    start_ix=txs_ix[0]
    end_ix=txs_ix[-1]
    
    
    # Generate edges to input to graph
    
    chunksize=len(txs)%ncpu
   
    #edges=chain.mapreduce_txes(extract_nodes_edges, lambda x: x , init=None, start=start_ix, end=end_ix, cpu_count=8)
    #edges=chain.mapreduce_txes(identity, identity, init=missing_param, start=start_ix, end=end_ix, cpu_count=8)

    '''
    with mp.Pool(processes=ncpu) as pool:
        edges=pool.map(extract_nodes_edges,txs,chunksize)
        
   
    '''
    for tx in txs:
        edges_i,nodes_i=extract_nodes_edges(tx)
        nodes.append(nodes_i)
        edges.append(edges_i)
        #print('Processed tx: {}'.format(tx.index))
       
    nodes=list(itertools.chain.from_iterable(nodes))
    edges=list(itertools.chain.from_iterable(edges))
    
    # Input to graph
    
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)
        

    #print('Generated Graph for Block starting at:{}'.format(init_block))
    return graph


In [43]:
# Function that receives a transaction and generates nodes and edges from addresses in transaction

def extract_nodes_edges(transaction):
    
    # Initialize values and get info from transaction
    edges=[]
    output_value=transaction.output_value
    block_height=transaction.block_height
    tx_id=transaction.index
    
    # Get inputs, types and values
    inputs=transaction.inputs.address
    #input_types=transaction.inputs.address.raw_type
    #input_block=transaction.inputs.address.first_tx.block.height
    input_val=transaction.inputs.value
    
    input_nodes=[(inp.address_num,{'raw_type':inp.raw_type,'block_created':inp.first_tx.block.height})for inp in inputs]
    

    # Get outputs and types
    outputs=transaction.outputs.address
    #output_types=transaction.outputs.address.raw_type
    #output_block=transaction.outputs.address.first_tx.block.height
    output_nodes=[(out.address_num,{'raw_type':out.raw_type,'block_created':out.first_tx.block.height})for out in outputs]
  
    # ****TODO: Add address balance as attribute to node**** 
    
    # Create nodes 
    
    nodes=input_nodes+output_nodes
    
    # Create edges (NetworkX will automatically create nodes when given edges)
  
    for i in range(len(inputs)):
        value=input_val[i]
        prop_value=value/len(outputs)
        
        for o in range(len(outputs)): 
            edge=(inputs[i].address_num,outputs[o].address_num,{'value':prop_value,'tx_id':block_height})
            edges.append(edge)
    
    return edges,nodes
            
            


In [44]:
# Test extract nodes_edges

test_tx=chain.tx_with_index(41847232)
edges,nodes=extract_nodes_edges(test_tx)
print(test_tx.hash)
print(nodes)
print(edges)


704f384862b91ff5d2cd170df55826f94c0d93f014d4ccf82406fabd61582434
[(19464725, {'raw_type': 2, 'block_created': 264954}), (40165049, {'raw_type': 2, 'block_created': 309015}), (30517405, {'raw_type': 2, 'block_created': 290181})]
[(19464725, 40165049, {'tx_id': 309015, 'value': 2500000.0}), (19464725, 30517405, {'tx_id': 309015, 'value': 2500000.0})]
